In [2]:
import pandas as pd
import math
import numpy as np
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from datetime import datetime
from IPython.display import display, HTML
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error


df_1_5=pd.read_csv('ura_1_5.csv',skiprows=1)
df_1_5=df_1_5[:-6] #Get rid of tail non-data description
display(df_1_5.tail(2))

df_6_10=pd.read_csv('ura_6_10.csv',skiprows=1)
df_6_10=df_6_10[:-6] #Get rid of tail non-data description
display(df_6_10.tail(2))

df_11_15=pd.read_csv('ura_11_15.csv',skiprows=1)
df_11_15=df_11_15[:-6] #Get rid of tail non-data description
display(df_11_15.tail(2))

df_16_20=pd.read_csv('ura_16_20.csv',skiprows=1)
df_16_20=df_16_20[:-6] #Get rid of tail non-data description
display(df_16_20.tail(2))

df_21_26=pd.read_csv('ura_21_26.csv',skiprows=1)
df_21_26=df_21_26[:-6] #Get rid of tail non-data description
display(df_21_26.tail(2))

df_27_28=pd.read_csv('ura_27_28.csv',skiprows=1)
df_27_28=df_27_28[:-6] #Get rid of tail non-data description
display(df_27_28.tail(2))

print("====================Final Dataframe=======================")
df = pd.concat([df_1_5, df_6_10,df_11_15,df_16_20,df_21_26,df_27_28], ignore_index=True)
display(df.tail(2))

df_dict=pd.read_csv('property_project.csv')
df_dict=df_dict[['address','lat','long']]
df_dict=df_dict.set_index('address').to_dict()
df_dict['lat']['JI XIANG COURT']

dates_dict={
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun":6,
    "Jul":7,
    "Aug":8,
    "Sep":9,
    "Oct":10,
    "Nov":11,
    "Dec":12    
}

# Preprocessing
# -Converting Floor Area(1400 to 1500 -> 1450)
# -Monthly rental/ sqft> Monthly rental Psf
# -Building/Project Name , District -> lat, long
# -Lease Commencement Date -> date
def g(x):
    if(x=='ANGULLIA PARK RESIDENCES @ ORCHARD'):
        return 'ANGULLIA PARK RESIDENCES@ORCHARD'
    return x
df['Building/Project Name'] = df['Building/Project Name'].apply(g)
    
    
def f(x):
    try:
        return sum([int(item) for item in str(x).split(' to ')])/2
    except:
        return 0
df['Floor Area (sq ft)'] = df['Floor Area (sq ft)'].apply(f)
df['Monthly Rent/ psf']=df['Monthly Gross Rent($)']/df['Floor Area (sq ft)']
df['lat']=df['Building/Project Name'].apply(lambda x:df_dict['lat'][x])
df['long']=df['Building/Project Name'].apply(lambda x:df_dict['long'][x])
def g(x):
    m,y=x.split('-')
    return pd.Timestamp(int(y),dates_dict[m],1).toordinal()
df['Lease Commencement Date']=df['Lease Commencement Date'].apply(g).astype(float)
# # Remove missing bedroom data
df['No. of Bedroom(for Non-Landed Only)']=df['No. of Bedroom(for Non-Landed Only)'].replace('na*',np.NaN)
df=df.dropna()
df = df.drop(df[df["Floor Area (sq ft)"]<50].index)

# # consolidaiton of Data
df=df[['Monthly Gross Rent($)','Monthly Rent/ psf','Postal District','lat','long','No. of Bedroom(for Non-Landed Only)','Floor Area (sq ft)','Lease Commencement Date']]
df['No. of Bedroom(for Non-Landed Only)']=df['No. of Bedroom(for Non-Landed Only)'].astype(float)
df['Floor Area (sq ft)']=df['Floor Area (sq ft)'].astype(float)

# KN : One Hot Encoding for district, One Hot Encoding for bedrooms
# KN : Refactor code to display data Prepocessing Step
df=df.rename(index=str, columns={"Monthly Gross Rent($)": "monthly_gross_rent", "Monthly Rent/ psf": "monthly_rent_psf","Postal District":"district","lat": "lat","long": "long","No. of Bedroom(for Non-Landed Only)": "no_bedroom","Floor Area (sq ft)": "floor_area","Lease Commencement Date": "date" })
print("====================Preprocessed Dataframe=======================")
display(df.tail(3))

,S/N,Building/Project Name,Street Name,Postal District,Type,No. of Bedroom(for Non-Landed Only),Monthly Gross Rent($),Floor Area (sq ft),Lease Commencement Date
35483,35484,MERAPRIME,JALAN BUKIT HO SWEE,3.0,Non-landed Properties,2,5000.0,800 to 900,Nov-2015
35484,35485,ONE SHENTON,SHENTON WAY,1.0,Non-landed Properties,2,5200.0,1000 to 1100,Nov-2015


,S/N,Building/Project Name,Street Name,Postal District,Type,No. of Bedroom(for Non-Landed Only),Monthly Gross Rent($),Floor Area (sq ft),Lease Commencement Date
53877,53878,RIVERGATE,ROBERTSON QUAY,9.0,Non-landed Properties,2,5600.0,1000 to 1100,Nov-2015
53878,53879,CITY SQUARE RESIDENCES,KITCHENER LINK,8.0,Non-landed Properties,4,5400.0,1500 to 1600,Nov-2015


,S/N,Building/Project Name,Street Name,Postal District,Type,No. of Bedroom(for Non-Landed Only),Monthly Gross Rent($),Floor Area (sq ft),Lease Commencement Date
59050,59051,VILLA MARINA,JALAN SEMPADAN,15.0,Non-landed Properties,3,3000.0,1200 to 1300,Nov-2015
59051,59052,D'MIRA,BOON TECK ROAD,12.0,Non-landed Properties,2,3200.0,900 to 1000,Nov-2015


,S/N,Building/Project Name,Street Name,Postal District,Type,No. of Bedroom(for Non-Landed Only),Monthly Gross Rent($),Floor Area (sq ft),Lease Commencement Date
44159,44160,LIVIA,PASIR RIS GROVE,18.0,Non-landed Properties,4,4000.0,1500 to 1600,Nov-2015
44160,44161,RIO VISTA,UPPER SERANGOON VIEW,19.0,Non-landed Properties,4,3100.0,1300 to 1400,Nov-2015


,S/N,Building/Project Name,Street Name,Postal District,Type,No. of Bedroom(for Non-Landed Only),Monthly Gross Rent($),Floor Area (sq ft),Lease Commencement Date
26456,26457,FORESQUE RESIDENCES,PETIR ROAD,23.0,Non-landed Properties,3,3200.0,1100 to 1200,Nov-2015
26457,26458,HILLINGTON GREEN,HILLVIEW AVENUE,23.0,Non-landed Properties,3,2850.0,1300 to 1400,Nov-2015


,S/N,Building/Project Name,Street Name,Postal District,Type,No. of Bedroom(for Non-Landed Only),Monthly Gross Rent($),Floor Area (sq ft),Lease Commencement Date
4679,4680,H2O RESIDENCES,FERNVALE LINK,28.0,Non-landed Properties,3,2700.0,1100 to 1200,Nov-2015
4680,4681,H2O RESIDENCES,FERNVALE LINK,28.0,Non-landed Properties,3,2500.0,1100 to 1200,Nov-2015


====================Final Dataframe=======================


,S/N,Building/Project Name,Street Name,Postal District,Type,No. of Bedroom(for Non-Landed Only),Monthly Gross Rent($),Floor Area (sq ft),Lease Commencement Date
223714,4680,H2O RESIDENCES,FERNVALE LINK,28.0,Non-landed Properties,3,2700.0,1100 to 1200,Nov-2015
223715,4681,H2O RESIDENCES,FERNVALE LINK,28.0,Non-landed Properties,3,2500.0,1100 to 1200,Nov-2015


====================Preprocessed Dataframe=======================


,monthly_gross_rent,monthly_rent_psf,district,lat,long,no_bedroom,floor_area,date
223713,2400.0,2.823529,27.0,1.438604,103.831283,2.0,850.0,735903.0
223714,2700.0,2.347826,28.0,1.392766,103.879232,3.0,1150.0,735903.0
223715,2500.0,2.173913,28.0,1.392766,103.879232,3.0,1150.0,735903.0


In [3]:
df1 = df[(df.no_bedroom>0.9)&(df.no_bedroom <1.1)]
display(df1.tail(3))

,monthly_gross_rent,monthly_rent_psf,district,lat,long,no_bedroom,floor_area,date
223675,1800.0,3.272727,28.0,1.386126,103.869516,1.0,550.0,735933.0
223685,2100.0,3.230769,28.0,1.386980,103.868809,1.0,650.0,735933.0
223698,1900.0,2.923077,28.0,1.386980,103.868809,1.0,650.0,735933.0


In [4]:
X=np.array(df1.drop(['monthly_gross_rent','monthly_rent_psf'],1))
Y=np.array(df1['monthly_rent_psf'])
X = preprocessing.scale(X)
X_train, X_test, y_train, y_test= train_test_split(X,Y,test_size=0.2,random_state=6)
rr = Ridge(alpha=0.01) # higher the alpha value, more restriction on the coefficients; low alpha > more generalization, coefficients are barely
# restricted and in this case linear and ridge regression resembles
rr.fit(X_train, y_train)
Ridge_train_score = rr.score(X_train,y_train)
Ridge_test_score = rr.score(X_test, y_test)
print("----Ridge Regression Training for 1 Bedroom----")
print("ridge regression train score low alpha:", Ridge_train_score) 
print("ridge regression test score low alpha:", Ridge_test_score)  

----Ridge Regression Training for 1 Bedroom----
ridge regression train score low alpha: 0.49189649747856456
ridge regression test score low alpha: 0.4674221184652754


In [5]:
y_predict = rr.predict(X_test)
y_predict = pd.DataFrame(y_predict)
y_test = pd.DataFrame(y_test)

squared_error = (y_predict - y_test) ** 2
mse = squared_error.mean()
rmse = mse ** (1/2)
print(rmse)

0    0.882301
dtype: float64


In [6]:
df2 = df[(df.no_bedroom>1.9)&(df.no_bedroom <2.1)]
display(df2.tail(3))

,monthly_gross_rent,monthly_rent_psf,district,lat,long,no_bedroom,floor_area,date
223707,2350.0,2.473684,27.0,1.410992,103.832387,2.0,950.0,735903.0
223710,2200.0,2.588235,28.0,1.392766,103.879232,2.0,850.0,735903.0
223713,2400.0,2.823529,27.0,1.438604,103.831283,2.0,850.0,735903.0


In [7]:
X=np.array(df2.drop(['monthly_gross_rent','monthly_rent_psf'],1))
Y=np.array(df2['monthly_rent_psf'])
X = preprocessing.scale(X)
X_train, X_test, y_train, y_test= train_test_split(X,Y,test_size=0.2,random_state=6)
rr = Ridge(alpha=0.01) # higher the alpha value, more restriction on the coefficients; low alpha > more generalization, coefficients are barely
# restricted and in this case linear and ridge regression resembles
rr.fit(X_train, y_train)
Ridge_train_score = rr.score(X_train,y_train)
Ridge_test_score = rr.score(X_test, y_test)
print("----Ridge Regression Training for 2 Bedroom----")
print("ridge regression train score low alpha:", Ridge_train_score) 
print("ridge regression test score low alpha:", Ridge_test_score)  

----Ridge Regression Training for 2 Bedroom----
ridge regression train score low alpha: 0.4102442134960934
ridge regression test score low alpha: 0.4113479740952154


In [8]:
y_predict = rr.predict(X_test)
y_predict = pd.DataFrame(y_predict)
y_test = pd.DataFrame(y_test)

squared_error = (y_predict - y_test) ** 2
mse = squared_error.mean()
rmse = mse ** (1/2)
print(rmse)

0    0.804509
dtype: float64


In [9]:
df3 = df[(df.no_bedroom>2.9)&(df.no_bedroom <3.1)]
display(df3.tail(3))

,monthly_gross_rent,monthly_rent_psf,district,lat,long,no_bedroom,floor_area,date
223708,2600.0,2.736842,27.0,1.438604,103.831283,3.0,950.0,735903.0
223714,2700.0,2.347826,28.0,1.392766,103.879232,3.0,1150.0,735903.0
223715,2500.0,2.173913,28.0,1.392766,103.879232,3.0,1150.0,735903.0


In [10]:
X=np.array(df3.drop(['monthly_gross_rent','monthly_rent_psf'],1))
Y=np.array(df3['monthly_rent_psf'])
X = preprocessing.scale(X)
X_train, X_test, y_train, y_test= train_test_split(X,Y,test_size=0.2,random_state=8)
rr = Ridge(alpha=0.01) # higher the alpha value, more restriction on the coefficients; low alpha > more generalization, coefficients are barely
# restricted and in this case linear and ridge regression resembles
rr.fit(X_train, y_train)
Ridge_train_score = rr.score(X_train,y_train)
Ridge_test_score = rr.score(X_test, y_test)
print("----Ridge Regression Training for 3 Bedroom----")
print("ridge regression train score low alpha:", Ridge_train_score) 
print("ridge regression test score low alpha:", Ridge_test_score)  

----Ridge Regression Training for 3 Bedroom----
ridge regression train score low alpha: 0.3010141536689477
ridge regression test score low alpha: 0.3010416267412743


In [11]:
y_predict = rr.predict(X_test)
y_predict = pd.DataFrame(y_predict)
y_test = pd.DataFrame(y_test)

squared_error = (y_predict - y_test) ** 2
mse = squared_error.mean()
rmse = mse ** (1/2)
print(rmse)

0    0.709164
dtype: float64


In [12]:
df4 = df[(df.no_bedroom>3.9)&(df.no_bedroom <4.1)]
display(df4.tail(3))

,monthly_gross_rent,monthly_rent_psf,district,lat,long,no_bedroom,floor_area,date
223684,3300.0,2.275862,27.0,1.424878,103.824647,4.0,1450.0,735933.0
223709,3280.0,2.116129,27.0,1.412497,103.835257,4.0,1550.0,735903.0
223712,3800.0,2.451613,27.0,1.410992,103.832387,4.0,1550.0,735903.0


In [13]:
X=np.array(df4.drop(['monthly_gross_rent','monthly_rent_psf'],1))
Y=np.array(df4['monthly_rent_psf'])
X = preprocessing.scale(X)
X_train, X_test, y_train, y_test= train_test_split(X,Y,test_size=0.2,random_state=8)
rr = Ridge(alpha=0.01) # higher the alpha value, more restriction on the coefficients; low alpha > more generalization, coefficients are barely
# restricted and in this case linear and ridge regression resembles
rr.fit(X_train, y_train)
Ridge_train_score = rr.score(X_train,y_train)
Ridge_test_score = rr.score(X_test, y_test)
print("----Ridge Regression Training for 4 Bedroom----")
print("ridge regression train score low alpha:", Ridge_train_score) 
print("ridge regression test score low alpha:", Ridge_test_score)  

----Ridge Regression Training for 4 Bedroom----
ridge regression train score low alpha: 0.248300550568617
ridge regression test score low alpha: 0.23509558339171066


In [14]:
y_predict = rr.predict(X_test)
y_predict = pd.DataFrame(y_predict)
y_test = pd.DataFrame(y_test)

squared_error = (y_predict - y_test) ** 2
mse = squared_error.mean()
rmse = mse ** (1/2)
print(rmse)

0    0.837733
dtype: float64


In [15]:
df5 = df[(df.no_bedroom>4.9)&(df.no_bedroom <5.1)]
display(df5.tail(3))

,monthly_gross_rent,monthly_rent_psf,district,lat,long,no_bedroom,floor_area,date
221591,4000.0,2.962963,27.0,1.280688,103.849207,5.0,1350.0,736450.0
222816,3500.0,2.592593,27.0,1.280688,103.849207,5.0,1350.0,736146.0
223041,4200.0,3.111111,27.0,1.280688,103.849207,5.0,1350.0,736085.0


In [16]:
X=np.array(df5.drop(['monthly_gross_rent','monthly_rent_psf'],1))
Y=np.array(df5['monthly_rent_psf'])
X = preprocessing.scale(X)
X_train, X_test, y_train, y_test= train_test_split(X,Y,test_size=0.2,random_state=8)
rr = Ridge(alpha=0.01) # higher the alpha value, more restriction on the coefficients; low alpha > more generalization, coefficients are barely
# restricted and in this case linear and ridge regression resembles
rr.fit(X_train, y_train)
Ridge_train_score = rr.score(X_train,y_train)
Ridge_test_score = rr.score(X_test, y_test)
print("----Ridge Regression Training for 5 Bedroom----")
print("ridge regression train score low alpha:", Ridge_train_score) 
print("ridge regression test score low alpha:", Ridge_test_score)  

----Ridge Regression Training for 5 Bedroom----
ridge regression train score low alpha: 0.0967578027291639
ridge regression test score low alpha: 0.09538441740223036


In [17]:
y_predict = rr.predict(X_test)
y_predict = pd.DataFrame(y_predict)
y_test = pd.DataFrame(y_test)

squared_error = (y_predict - y_test) ** 2
mse = squared_error.mean()
rmse = mse ** (1/2)
print(rmse)

0    1.170465
dtype: float64
